# Running SQL with the Db2 Data Management Console

This Jupyter Notebook contains examples of how to use the Open APIs to run SQL and use Visual Exaplain in the Db2 Data Management Console. 

### Import Helper Classes
For more information on these classes, see the Lab on Db2 Data Management Console Overview

In [ ]:
%run ./dmc_setup.ipynb

### Db2 Data Management Console Connection
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [ ]:
# Connect to the Db2 Data Management Console service
Console  = 'http://localhost:11080'
profile  = 'SAMPLE'
user     = 'DB2INST1'
password = 'db2inst1'

# Set up the required connection
profileURL = "?profile="+profile
databaseAPI = Db2(Console+'/dbapi/v4')
databaseAPI.authenticate(user, password, profile)
database = Console

### Confirm the connection
To confirm that your connection is working you can check the status of the moitoring service. You can also check your console connection to get the details of the specific database connection you are working with. Since your console user id and password are may be limited as to which databases they can access you need to provide the connection profile name to drill down on any detailed information for the database.

In [ ]:
# List Monitoring Profile
r = databaseAPI.getProfile(profile)
json = databaseAPI.getJSON(r)
print(json)

### Running SQL
You can use the console API to run single SQL statement or sets of statements in a single call. In the following example we run three statements in a single API call. When the statement is started we get a run handle we can use to access the results of the SQL execution. This example runs each statement five times. 

In [ ]:
#Run SQL Statements
runtimes = 5
sqlList = ['select SYSIBM.SYSTABLES.* from SYSIBM.SYSTABLES, SYSIBM.SYSCOLUMNS,SYSIBM.SYSINDEXES,SYSIBM.SYSVIEWS,SYSIBM.SYSVIEWDEP,SYSIBM.SYSPLAN,SYSIBM.SYSPLANDEP,SYSIBM.SYSSTMT,SYSIBM.SYSPLANAUTH',
           'select * from SYSIBM.SYSINDEXES','select * from syscat.tables', 
           'Select * from syscat.indexes']
runIndex = []
for x in range(0, runtimes):
    for sqlText in sqlList:
        r = databaseAPI.runSQL(sqlText)
        statusCode = databaseAPI.getStatusCode(r);
        if (databaseAPI.getStatusCode(r)==201):
            print(databaseAPI.getJSON(r)['id']+': running')
            runIndex.append(databaseAPI.getJSON(r)['id'])     
print('Done')

You can then cycle through the list of run handles and collect the results of each run.

In [ ]:
#Collect the run results of each statement into an array
indexLength = len(runIndex)
jsonArray = []
for x in range(0, indexLength ):
    r = databaseAPI.getSQLJobResult(runIndex[x])
    jsonArray.append(databaseAPI.getJSON(r))

For each statement we collect the runtime can convert to ms. We can then collect a dataframe to show how each statement performs over several iterations. 

In [ ]:
resultsDF = pd.DataFrame(columns=['SQL Text', 'Runtime'])
for x in range(0, indexLength):
    runtime = jsonArray[x]['results'][0]['runtime_seconds'] * 1000
    sql = jsonArray[x]['results'][0]['command']
    s = pd.Series([sql,runtime], index=['SQL Text','Runtime'])
    resultsDF = resultsDF.append(s, ignore_index=True)    
display(resultsDF.sort_values(by=['SQL Text','Runtime']))

We can then collect the most recent package cache information. 

**Note:** There is a delay while the Db2 Data Management Console collects this information. The following script may not find any rows immediately.

In [ ]:
# Retrieve the current package cache list 
# Show the first ten as sorted by the statement execution time
r = databaseAPI.getCurrentPackageCacheList("false")
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:  
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Available Data Frame Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='stmt_exec_time_ms', ascending=False)
        display(df[['stmt_text','stmt_exec_time_ms','stmtid']].head(100))
    else: 
        print('No data returned')  
else:
    print(databaseAPI.getStatusCode(r))

Using one statement we can compare the statements we ran with the contents of the package cache to just see the performance of the statements we ran and filter out any other statements. 

In [ ]:
try:
    display(df[['stmt_text','stmt_exec_time_ms','stmtid']].loc[df['stmt_text'].isin(sqlList)])
except:
    print("No data available")

You can always also use the console user interface to see the latest statements in the package cache.

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/package_cache'+profileURL, width=1400, height=480)

### Analyzing Statements
You can use both the microservices built into the console as well as the monitoring APIs to anaylze the performance of single SQL statement or identify statement that need your attention. You can visually explain any statement by calling the explain/create service and imbed the interactive interface in any IFrame. **Note:** This may take a minute to run.

In [ ]:
# Visually explain the access plan for an SQL Statement
SQLStatement = 'select * from syscat.tables'
IFrame(database+'/console/?mode=compact#sql/explain/create/'+SQLStatement+profileURL, width=1400, height=480)

You can also track which statements are running right now using either the microservice User Interface or a direct API call.

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/inflight_executions'+profileURL, width=1400, height=360)

In [ ]:
# Retrieve the current statements running now
# Display the top 10 by execution time
r = databaseAPI.getInflightCurrentList()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='exec_time_ms', ascending=False)
        display(df[['application_name','stmt_text','exec_time_ms','estimated_runtime_ms']].head(10))
    else:
        print('No data returned')
else:
    code = databaseAPI.getStatusCode(r)
    databaseAPI.printResponse(r, code)

If you have individual statement monitoring enabled you can see a list of all the statements that ran on your system either through a micro service UI or an API call.

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/individual'+profileURL, width=1400, height=480)

In [ ]:
# Retrieve the statements that ran over the last day.
# Retrieve the first 10000 statements
# Show the top ten by Total CPU TIME
startTimeMinusWeek = endTime - oneDay
r = databaseAPI.getIndividualStatementExecution(startTimeMinusWeek, endTime, 10000)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0:     
        df = pd.DataFrame(json_normalize(json['resources']))
        print('Columns')
        print(', '.join(list(df)))
        df = df.sort_values(by='total_cpu_time', ascending=False)
        display(df[['sql_text','total_cpu_time','wlm_queue_time_total','stmt_exec_time']].head(10))
    else: 
        print('No data returned')  
else:
    code = databaseAPI.getStatusCode(r)
    databaseAPI.printResponse(r, code)

In one last example you can take a higher level look at the applications connected to your database to see which application is driving the most work.

In [ ]:
IFrame(database+'/console/?mode=compact#monitor/connections'+profileURL, width=1400, height=480)

In [ ]:
# Display the 10 most recently started Database Connections
r = databaseAPI.getCurrentApplicationsConnections()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    if json['count'] > 0: 
        df = pd.DataFrame(json_normalize(json['resources']))
        print(', '.join(list(df)))
        df = df.sort_values(by='connection_start_time', ascending=False)
        df['connection_start_time'] = df['connection_start_time'].apply(epochtotimeseries)
        display(df[['application_name','application_handle','connection_start_time']].head(10))
    else: 
        print('No data returned')  
else:
    print(databaseAPI.getStatusCode(r))

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]